In [1]:
from pytezos import pytezos

In [2]:
# TODO: move this to settings.py
settings = dict(
    SHELL_URL = 'https://edonet-tezos.giganode.io/',
    # SHELL_URL = 'https://edonet.smartpy.io',
    CONTRACT_ADDRESS = 'KT1HbiznMedpC5BcQnUCUd5zCaXcemsU25Sk',
    KEYS_DIRECTORY = 'test-keys',

    # default storage used to deploy new contract:
    DEFAULT_INITIAL_CONTRACT_STORAGE = {
        'betsAgainstLedger': 0,
        'betsAgainstSum': 0,
        'betsForLedger': 0,
        'betsForSum': 0,
        'closedRate': 0,
        'closedTime': 0,
        'currencyPair': 'XTZ-USD',
        'isBetsForWin': False,
        'isClosed': False,
        'oracleAddress': 'KT1RCNpUEDjZAYhabjzgz1ZfxQijCDVMEaTZ',
        'targetRate': 0,
        'targetTime': 0
    },

    IS_ASYNC_ENABLED = False,
)

In [11]:
from os.path import join
from os import listdir
from pytezos import pytezos
from pprint import pprint


def load_key_filenames(directory):

    def make_key_name(fn):
        return fn.split('.')[0]

    filenames =  {
        make_key_name(fn): join(directory, fn)
        for fn in listdir(directory) if fn.lower().endswith('.json')
    }

    if not filenames:
        raise Exception(
            'Please add test keys into pytezos-jupyter-present/test-keys directory '
            + '(you can use https://faucet.tzalpha.net/')

    return filenames


class CrystalOperator:
    """ Operates with deployed on tezos blockchain smart contract
        using multiple pytezos instances with different keys
    """

    pass

# crystal = CrystalOperator(settings)

In [18]:
from pytezos.operation.result import OperationResult


class KeysManager:
    def __init__(self, settings):

        self.settings = settings
        self.key_filenames = load_key_filenames(settings['KEYS_DIRECTORY'])
        self.shell_url = settings['SHELL_URL']
        self.is_async_enabled = settings['IS_ASYNC_ENABLED']

        self.pytezos_instances = {
            key_name: pytezos.using(key=key_filename, shell=self.shell_url)
            for key_name, key_filename in self.key_filenames.items()
        }

        assert len(self.pytezos_instances)
        print(f'Successfully loaded {len(self.pytezos_instances)} pytezos keys:')
        [print(f'- {key_name}') for key_name in self.pytezos_instances]


    def activate_keys(self):
        """ Runs activate_account for each loaded key """

        for pt in self.pytezos_instances:
            try:
                pt.activate_account().autofill().sign().inject(_async=self.is_async_enabled)
                pt.reveal().autofill().sign().inject(_async=self.is_async_enabled)
            except Exception as e:
                print(f'Error: {type(e)}, "{e}"')


class ContractManager:

    def __init__(self, pytezos, settings):
        self.settings = settings
        self.contract_address = settings['CONTRACT_ADDRESS']
        self.is_async_enabled = settings['IS_ASYNC_ENABLED']
        self.pytezos = pytezos
        self.origin_contract = self.pytezos.contract(self.contract_address)


    def create_new_storage(self, **kwargs):
        """ Creates new storage for Fortune Crystal Ball smart contract with
            custom storage params in kwargs
        """

        storage = self.settings['DEFAULT_INITIAL_CONTRACT_STORAGE'].copy()
        storage.update(kwargs)
        return storage


    def find_originated_contract_address(self, new_contract_result):
        """ Searches for new originated contract address in blockchain """

        op_hash, branch = new_contract['hash'], new_contract['branch']
        blocks = cm.pytezos.shell.blocks[branch:]
        opg = blocks.find_operation(op_hash)
        res = OperationResult.from_operation_group(opg)
        originated_contract_address = res[0].originated_contracts[0]
        return originated_contract_address


    def deploy_new_contract(self, **kwargs):
        """ Deploys new contract with params transfered in kwargs """

        new_storage = self.create_new_storage(**kwargs)
        print(f'Deploying new contract with storage:')
        pprint(new_storage)

        new_contract = self.pytezos.origination(
            script=self.origin_contract.script(initial_storage=new_storage))
        new_contract = new_contract.autofill().sign().inject(_async=self.is_async_enabled)

        originated_contract_address = self.find_originated_contract_address(new_contract)
        print(f'Contract successfully originated at address: {originated_contract_address}')

        # return new contract manager with replaced contract address:
        new_settings = self.settings.copy()
        new_settings.update(CONTRACT_ADDRESS=originated_contract_address)

        return ContractManager(self.pytezos, new_settings)


class CrystalContractManager(ContractManager):
    def bet(self):
        pass

keys = KeysManager(settings)

# If your keys is not activated, you can activate it by running:
# keys.activate_keys()

cm = ContractManager(keys.pytezos_instances['tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE'], settings)

Successfully loaded 5 pytezos keys:
- tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE
- tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM
- tz1MdaJfWzP5pPx3gwPxfdLZTHW6js9havos
- tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ
- tz1ZAzDvkZCT2LAyPN8Kdxw3kes7xfWerZhZ


In [ ]:
cm

In [20]:
new_contract = cm.deploy_new_contract(targetRate=4, targetTime=1)

Deploying new contract with storage:
{'betsAgainstLedger': 0,
 'betsAgainstSum': 0,
 'betsForLedger': 0,
 'betsForSum': 0,
 'closedRate': 0,
 'closedTime': 0,
 'currencyPair': 'XTZ-USD',
 'isBetsForWin': False,
 'isClosed': False,
 'oracleAddress': 'KT1RCNpUEDjZAYhabjzgz1ZfxQijCDVMEaTZ',
 'targetRate': 4,
 'targetTime': 1}
Wait 10 seconds until block BMKJbPeTATMmTPCsvfjLeNXHS426Wb8wawsjNTCvzpR65A1K5NS is finalized


2021-02-28 13:06:11.603 | DEBUG    | pytezos.rpc.search:find_operation:206 - checking level 46250...


In [38]:
contract = cm.pytezos.contract('KT1V1x3nwbFU2WatcV6hBmCmCMdDB6E7BwLE')

In [ ]:
contract.close()

In [39]:
contract.betAgainst().with_amount(414_000).as_transaction().autofill().sign().inject(_async=False)

Wait 30 seconds until block BMMB9AmSaw52voNLp4sog5GySmpqH688Cw79pKgFLuJVRaCrgXZ is finalized


2021-02-28 13:20:56.694 | DEBUG    | pytezos.rpc.search:find_operation:206 - checking level 46278...


{'protocol': 'PtEdo2ZkT9oKpimTah6x2embF25oss54njMuPzkJTEi5RqfdZFA',
 'chain_id': 'NetXSgo1ZT2DRUG',
 'hash': 'opUAyvDPuzf7RYg8CXeqhtTZnnCNoJqfxhoMZgFSFHno1RRRDiU',
 'branch': 'BLGq1Fff4bDVHyCs6XuZe8e9s74y2sogP1WkJfrzgHw2KeqEHqW',
 'contents': [{'kind': 'transaction',
   'source': 'tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE',
   'fee': '1767',
   'counter': '93018',
   'gas_limit': '14925',
   'storage_limit': '71',
   'amount': '414000',
   'destination': 'KT1V1x3nwbFU2WatcV6hBmCmCMdDB6E7BwLE',
   'parameters': {'entrypoint': 'betAgainst', 'value': {'prim': 'Unit'}},
   'metadata': {'balance_updates': [{'kind': 'contract',
      'contract': 'tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE',
      'change': '-1767'},
     {'kind': 'freezer',
      'category': 'fees',
      'delegate': 'tz1R55a2HQbXUAzWKJYE5bJp3UvvawwCm9Pr',
      'cycle': 22,
      'change': '1767'}],
    'operation_result': {'status': 'applied',
     'storage': [[{'prim': 'Pair',
        'args': [{'prim': 'Pair',
          'args': [{'i

### TODO:
+ сделать 5 ключей с помощью кранов
- сделать небольшой интерфейс для взаимодействия со смарт контрактом
    + выпуск нового смарт контракта на основе имеющегося
    - различные взамодействия с контрактом
    - написать пару тестов
- сделать красивую презентацию, с маркдауном, с графикой (распределение ставок например сделать в графиках)

In [40]:
contract.betAgainst().with_amount(414_000).as_transaction().autofill().sign().inject(_async=False)

MichelsonError: ({'id': 'proto.008-PtEdo2Zk.michelson_v1.script_rejected',
  'kind': 'temporary',
  'location': 123,
  'with': {'string': 'Account already made betAgainst'}},)

In [41]:
contract.close().as_transaction().autofill().sign().inject(_async=False)

Wait 13 seconds until block BLrZusve2w715mzKzShFCpta2GT7JKfRMQ1eii7eicPa8uhX5Bv is finalized


2021-02-28 13:21:51.744 | DEBUG    | pytezos.rpc.search:find_operation:206 - checking level 46280...


{'protocol': 'PtEdo2ZkT9oKpimTah6x2embF25oss54njMuPzkJTEi5RqfdZFA',
 'chain_id': 'NetXSgo1ZT2DRUG',
 'hash': 'onznQyu8Z97xfzsD3TzGg8vKhnxZVVsSsvmRSLFZZ4WNjExmCaL',
 'branch': 'BKumNwnPnAxErgwwamK4Moq6CfGVt4hqBGfpx3ePNx3pMqihQp6',
 'contents': [{'kind': 'transaction',
   'source': 'tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE',
   'fee': '9696',
   'counter': '93019',
   'gas_limit': '94286',
   'storage_limit': '7',
   'amount': '0',
   'destination': 'KT1V1x3nwbFU2WatcV6hBmCmCMdDB6E7BwLE',
   'parameters': {'entrypoint': 'close', 'value': {'prim': 'Unit'}},
   'metadata': {'balance_updates': [{'kind': 'contract',
      'contract': 'tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE',
      'change': '-9696'},
     {'kind': 'freezer',
      'category': 'fees',
      'delegate': 'tz1VpvtSaSxKvykrqajFJTZqCXgoVJ5cKaM1',
      'cycle': 22,
      'change': '9696'}],
    'operation_result': {'status': 'applied',
     'storage': [[{'prim': 'Pair',
        'args': [{'prim': 'Pair',
          'args': [{'int': '14503

In [ ]:
crystal_ball.withdraw().as_transaction().autofill().sign().inject(_async=False)